[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_05_seleccion_modelos-published.ipynb)

# Selección de modelos

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display


## Cross validation

Hasta ahora sólo habíamos visto (ver en el [notebook 03](https://github.com/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_03_arboles_de_decision_sklearn-published.ipynb)) que ibamos a dividir los datos en train y test.


En esta semana vimos la opción de hacer validación cruzada. En esta oportunidad lo que haremos sera realizar una exploración de hiperparámetros para para árboles incorporando conceptos de la clase de esta semana.
Vamos a experimentar usando k-fold (con k=10) para explorar distintos valores de configuración de `DecisionTreeClassifier` para seleccionar el hiperparámetro que nos parezca el mejor.
Ensayaremos áltura máxima con valores `[None, 1, 2, 3, 5, 8, 13, 21]`.

Nos interesará:
- controlar el tiempo de entrenamiento
- generar alguna métrica que elijamos para seleccionar la áltura máxima

Con la mejor configuración obtenida entrenar un clasificador con todos los datos de desarrollo.
    
Evaluar el comportamiento con el set de evaluación
    



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
import timeit
import pandas as pd

def cargar_datos():
    df = pd.read_csv('https://github.com/aprendizaje-automatico-dc-uba-ar/material/raw/main/dataset/data_05/seleccion_modelos.csv')
    X = df.drop("Y", axis=1)
    y = df.Y
    return X.to_numpy(), y.to_numpy()

X, y = cargar_datos()
X,y

(array([[ 0.74946762, -1.83875845,  2.31697643, ...,  0.38502105,
          1.15910799,  0.36490854],
        [ 1.36142303,  0.17739336, -1.06308644, ..., -0.00426734,
         -1.63632588, -0.8335227 ],
        [ 0.12238178,  1.03817562, -1.46411856, ...,  1.69000604,
         -0.57898546,  0.34605186],
        ...,
        [ 0.77302083,  0.76832206, -0.36434009, ..., -0.05485574,
         -0.51528272,  0.7993889 ],
        [-0.54238642, -0.87839139,  0.68624112, ...,  0.20799802,
          1.06110671, -0.34658297],
        [-0.03135099,  0.93928815, -1.16413366, ...,  0.73422269,
         -0.37504853, -0.59041732]]),
 array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
   

Primero separaremos nuestro data set entre **desarrollo** y **evaluación** en un 10%. Para esto podemos usar `train_test_split`

In [ ]:
# separamos entre dev y eval
X_dev, X_eval, y_dev, y_eval = train_test_split(
                    X,
                    y,
                    random_state=4,
                    test_size=0.1)
type(y_eval)

numpy.ndarray

Por el momento dejaremos el set de evaluación de lado y nos manejaremos con el de desarrollo.

Pasemos a experimentar los distinos `h_max` posibles.

Usaremos estas dos funciones para entrenar un árbol y para usarlo para predecir respectivamente:

In [ ]:
def train_tree(X_tr: np.ndarray, y_tr: np.ndarray, tree_params={}) -> DecisionTreeClassifier:
    arbol = DecisionTreeClassifier(**tree_params)
    arbol.fit(X_tr, y_tr)

    return arbol

def tree_predict(ab: DecisionTreeClassifier, X_test: np.ndarray) -> np.ndarray:
    predictions = ab.predict(X_test)
    return predictions

Y definimos la métrica a usar. A modo de ejemplo figura accuracy.

Cambiar la medida por una nueva vista en clase.

In [ ]:
from typing import Tuple, Any

In [ ]:
def accuracy(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    TP_TN = sum([y_i == y_j for (y_i, y_j) in zip(y_predicted, y_real)])
    P_N = len(y_real)
    return TP_TN /P_N
### Nuevo

def confusion_matrix(y_real: list, y_predicted: list, positive_label: Any, show: bool =False) -> Tuple[int, int, int, int]:
    tp = sum([((y_i == y_j) and (y_i == positive_label)) for (y_i, y_j) in zip(y_predicted, y_real)])
    tn = sum([((y_i == y_j) and (y_i != positive_label)) for (y_i, y_j) in zip(y_predicted, y_real)])
    fp = sum([((y_i != y_j) and (y_i == positive_label)) for (y_i, y_j) in zip(y_predicted, y_real)])
    fn = sum([((y_i != y_j) and (y_i != positive_label)) for (y_i, y_j) in zip(y_predicted, y_real)])

    if show:
        display(pd.DataFrame([[tp, fn], [fp, tn]], index=["real +", "real -"], columns=["pred +", "pred -"]))
    return tp, tn, fp, fn


def accuracy_score(tp: int, tn: int, fp: int, fn: int) -> float:
    return (tp+tn)/(tp+tn+fp+fn)


def precision_score(tp: int, tn: int, fp: int, fn: int) -> float:
    if (tp+fp) != 0:
      return tp/(tp+fp)
    else:
      return 0


def recall_score(tp: int, tn: int, fp: int, fn: int) -> float:
    if (tp+fn) != 0:
      return tp/(tp+fn)
    else:
      return 0


def f_beta_score(tp: int, tn: int, fp: int, fn: int, beta: float) -> float:
    prec = precision_score(tp, tn, fp, fn)
    recl = recall_score(tp, tn, fp, fn)
    if ((beta**2)*prec+recl) != 0:
      return (1+beta**2)*prec*recl/((beta**2)*prec+recl)
    else:
      return 0


def f1_score(tp: int, tn: int, fp: int, fn: int) -> float:
    return f_beta_score(tp, tn, fp, fn, beta=1)


def metrica_seleccionada(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    tp, tn, fp, fn = confusion_matrix(y_real=y_real, y_predicted=y_predicted, positive_label=1)
    #return accuracy(y_predicted,y_real)
    #return f1_score(tp,tn,fp,fn)
    return f1_score(tp,tn,fp,fn)

Realización del experimento.

Nota: se inicializa con una semilla para poder reproducir el resultado.

In [ ]:
results = []

np.random.seed(44)
for h_max in [None, 1, 2, 3, 5, 8, 13, 21]:
    kf = KFold(n_splits=10)
    y_pred = np.empty(y_dev.shape)
    y_pred.fill(np.nan)

    # generamos para cada fold una predicción
    for train_index, test_index in kf.split(X_dev):

        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = X_dev[train_index], X_dev[test_index]
        kf_y_train, kf_y_test = y_dev[train_index], y_dev[test_index]

        current_tree = train_tree(kf_X_train, kf_y_train,
                                    tree_params={"max_depth":h_max})
        predictions = tree_predict(current_tree, kf_X_test)
        y_pred[test_index] = predictions

    current_score = metrica_seleccionada(y_pred, y_dev)

    results.append((h_max,current_score))


# Ordenamos los resultados (puede ser que convenga del derecho o del reves)
r = sorted(results, key=lambda x: x[1], reverse=True)

print("Órden obtenido según la métrica elegida")
for idx, (h, sc) in enumerate(r):
    print(f"\t{idx+1}- h_max={h} con {sc:.3f}")


Órden obtenido según la métrica elegida
	1- h_max=1 con 0.833
	2- h_max=3 con 0.821
	3- h_max=2 con 0.809
	4- h_max=5 con 0.808
	5- h_max=8 con 0.777
	6- h_max=None con 0.755
	7- h_max=21 con 0.738
	8- h_max=13 con 0.727


Con los resultados obtenidos podemos elegir la `h_max` que nos parezca mejor. Con eso vamos a reentrenar el modelo con todos los datos.

¿Qué teníamos que tener en cuenta en estos casos?

In [ ]:
# elegimos
h_max = 1 # COMPLETAR
selection_score = 0.833 # COMPLETAR

In [ ]:
assert selection_score is not None, "Completar la celda anterior para continuar"

Con el mejor parámetro entrenamos un nuevo clasificador:

In [ ]:
print(f"Construimos nuestro clasificador con parámetro 'max_depth'={h_max}."
     + f"Para seleccionarlo el score que habíamos obtenido era {selection_score:.3f}")

best_tree = train_tree(X_dev, y_dev,
                            tree_params={"max_depth":h_max})


Construimos nuestro clasificador con parámetro 'max_depth'=1.Para seleccionarlo el score que habíamos obtenido era 0.833


Podemos evaluar este árbol en train.

In [ ]:
y_pred = tree_predict(best_tree, X_dev)
best_tree_score = metrica_seleccionada(y_pred, y_dev)

¿Qué nos están diciendo estos dos scores?¿Para qué nos sirven?

In [ ]:
best_tree_score

0.8631578947368421

Por única vez vemos como funciona nuestro entrenamiento en los datos de **evaluación** que no habíamos mirado.

In [ ]:
y_pred_eval = tree_predict(best_tree, X_eval)
best_tree_score_eval = metrica_seleccionada(y_pred_eval, y_eval)

print(f"Con el árbol entrenado con el parámetro seleccionado tenemos en eval un score de {best_tree_score_eval:.3f}")

Con el árbol entrenado con el parámetro seleccionado tenemos en eval un score de 0.600


In [ ]:
EVAL = best_tree_score_eval
CV = best_tree_score
print((((EVAL - CV) / CV ) * 100))

-30.487804878048784


In [ ]:
EVAL

0.6

## Opcionales

1. Simular qué hubiese ocurrido si hubieramos elegido un K distinto. ¿La diferencia entre el score en *dev* y el score en *eval* cambia significativamente?
2. Repetir el mismo ejercicio de elegir la mejor combinación de parametros pero esta vez establecer una grilla donde se exploren al menos dos hiperparámetros que no sean la altura máxima. Revisar la documentación de `DecisionTreeClassifier`, por ejemplo pueden elegir la **medida de impureza** y el **mínimo de muestas necesario para realizar un split**. Definir los rangos necesarios para explorar más de un valor de cada hiperparámetro considerado. ¿Este modelo fue mejor que el obtenido en el punto anterior?

**Importante**: en este punto nos tomamos la licencia de usar nuevamente el conjunto de evaluación. El re-uso de el conjunto de evaluación sólo lo permitimos en este caso por motivos pedagócios. Pero **NO DEBE** suceder en la práctica.

